# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
adult_dt

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [2]:
from sklearn.model_selection import train_test_split
X = adult_dt.drop('income', axis=1)
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)

X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

1. The random state is a parameter in the train_test_split function that controls the shuffling of the data before splitting it into training and testing sets. I set the random state to 42.

2. Ensures the data split is consistent every time the code is run, making results reproducible. Allows fair comparison of different models or configurations by using the same train-test split. Also, ensures performance metrics are due to model changes, not differences in data splits.

*(Comment here.)*

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline

# Assume numerical_features and categorical_features lists are defined with the respective column names
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define the transformers for numerical and categorical features
num_transforms = [
    ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('robust_scaler', RobustScaler())
]

cat_transforms = [
    ('simple_imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
]

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=num_transforms), numerical_features),
        ('cat', Pipeline(steps=cat_transforms), categorical_features)
    ]
)

print(preprocessor)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot_encoder',
                                                  OneHotEncoder(drop='if_binary',
                                                              

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier())
])

# Print the model pipeline to verify
print(model_pipeline)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robust_scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [5]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Define the scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': make_scorer(balanced_accuracy_score)
}

# Perform cross-validation
cv_results = cross_validate(
    model_pipeline,
    X_train,
    Y_train,
    cv=5,
    scoring=scoring,
    return_train_score=True
)

# Report the training and validation results
for metric in scoring.keys():
    train_score_mean = np.mean(cv_results[f'train_{metric}'])
    train_score_std = np.std(cv_results[f'train_{metric}'])
    test_score_mean = np.mean(cv_results[f'test_{metric}'])
    test_score_std = np.std(cv_results[f'test_{metric}'])
    
    print(f"Training {metric}: {train_score_mean:.4f} (+/- {train_score_std:.4f})")
    print(f"Validation {metric}: {test_score_mean:.4f} (+/- {test_score_std:.4f})\n")



Training neg_log_loss: -0.0816 (+/- 0.0005)
Validation neg_log_loss: -0.3676 (+/- 0.0121)

Training roc_auc: 1.0000 (+/- 0.0000)
Validation roc_auc: 0.9036 (+/- 0.0019)

Training accuracy: 1.0000 (+/- 0.0000)
Validation accuracy: 0.8547 (+/- 0.0045)

Training balanced_accuracy: 1.0000 (+/- 0.0001)
Validation balanced_accuracy: 0.7765 (+/- 0.0046)



Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [6]:
# Create a DataFrame from the cross-validation results
cv_results_df = pd.DataFrame(cv_results)

# Sort the DataFrame by the negative log loss of the test set
cv_results_df_sorted = cv_results_df.sort_values(by='test_neg_log_loss', ascending=False)

# Display the sorted DataFrame
print(cv_results_df_sorted)


    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
3  11.507299    0.231758          -0.351676           -0.081775      0.906250   
0  10.746332    0.181081          -0.361751           -0.082146      0.905365   
2  11.807943    0.173121          -0.362661           -0.080855      0.901529   
1  10.602421    0.188578          -0.375102           -0.081210      0.902175   
4  11.114278    0.275061          -0.386761           -0.081819      0.902553   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
3            1.0       0.861562        1.000000                0.784632   
0            1.0       0.849967        1.000000                0.773435   
2            1.0       0.853006        1.000000                0.772524   
1            1.0       0.850625        0.999945                0.773255   
4            1.0       0.858271        0.999945                0.778460   

   train_balanced_accuracy  
3                 1.000000  
0   

Calculate the mean of each metric. 

In [7]:
# Calculate the mean of each metric
mean_results = cv_results_df.mean()

# Print the mean values
print("\nMean of each metric:")
print(mean_results)



Mean of each metric:
fit_time                   11.155655
score_time                  0.209920
test_neg_log_loss          -0.367590
train_neg_log_loss         -0.081561
test_roc_auc                0.903575
train_roc_auc               1.000000
test_accuracy               0.854686
train_accuracy              0.999978
test_balanced_accuracy      0.776461
train_balanced_accuracy     0.999955
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [9]:
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Fit the model pipeline on the training data
model_pipeline.fit(X_train, Y_train)

# Get prediction probabilities
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Get predicted classes
Y_pred = model_pipeline.predict(X_test)

# Calculate performance metrics
test_metrics = {
    'neg_log_loss': log_loss(Y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, Y_pred)
}

# Display the results as a dictionary
print(test_metrics)


{'neg_log_loss': 0.3872464439280904, 'roc_auc': 0.8999369307171174, 'accuracy': 0.852492578564848, 'balanced_accuracy': 0.7704392300214424}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

1. Binary Classification Compatibility: scikit-learn models expect binary target variables to be numeric (0 and 1). This makes it straightforward to use the target variable in classification models.

2. Performance Metrics: Metrics like ROC AUC and log loss require the target variable to be numeric. Recoding simplifies their calculation.

3. Consistency: Ensures the target variable format is consistent throughout the data pipeline, reducing the risk of errors.

4. Simplifies Training: Numeric target variables are directly usable by scikit-learn estimators, streamlining the model training process.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.